# Overview

### This code is used to Scrape data from the county assessors website.  It gets all the tax/ assesor appraisals data for each of the homes within out dataset.   I use the data extensively to predict the value of our homes.

In [1]:
# import required libraries
from bs4 import BeautifulSoup
import pandas as pd # data analysis
import requests as rq # get url
import re
from bs4 import BeautifulSoup as bs # data scraping

# Scraping

In [2]:
# read file stored in current working directory, using excel file here
data = pd.read_csv("./data/kc_house_data.csv")

In [3]:
# creates dataframes for the scraped data
Table1 = pd.DataFrame(columns = ["ParcelNumber", "Parcel_Number", "Name", "Address", "Legal", 
                                 "YearBuilt", "TotalSqFootage", "NumBedrms", "NumBthrms", 
                                 "Grade", "Condition", "LotSize", "Views","Waterfront"])

TaxRollTable = pd.DataFrame(columns = 
                            ["ParcelNumber", "Valueyear", "TaxYear", "AppraisedLandValue", 
                             "AppraisedImpsValue", "AppraisedTotal",
                                      "Appraised Imps Increase","Taxable Land Value", 
                             "Taxable Imps Value", "Taxable Total"])

In [49]:
# Scraping function, needed to run this function several times given restrictions imposed by assesor website 
headers = {
        'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/86.0.4240.111 Safari/537.36'}


for parcelNumber in data["id"]:
    ParcelNumber = parcelNumber
    norecordfound = False
    # ParcelNumber = "7338200240"
    r=rq.get('https://blue.kingcounty.com/Assessor/eRealProperty/Dashboard.aspx?ParcelNbr={}'.format(ParcelNumber))
    p=BeautifulSoup(r.text,'html.parser')
    
    if p.find("span", id="cphContent_LblMsg"):
        checkfornodata = p.find("span" ,id="cphContent_LblMsg")
        if checkfornodata is not None:
            if checkfornodata.text == "No data found.":
                norecordfound = True
                Table1.loc[len(Table1)] = [ParcelNumber, "0", "NoMatch", "0", "0", "0","0","0", "0", "0","0","0","0","0"]
                #print (nodata.text) 
                continue
            
    if not norecordfound:          
#GET DATA FROM TABLE 1--------------------------------------------------------
        theRow = []
        theRow.append(ParcelNumber)
        if p.find('td', text='Parcel Number') !=-1:
            Parcel_Number = p.find('td', text='Parcel Number').find_next('td').text.strip()
        else:
            Parcel_Number = "999999999"
        #print(Parcel_Number)
        theRow.append(Parcel_Number)

        if p.find('td', text='Name') !=-1:
            Name = p.find('td', text='Name').find_next('td').text.strip()
        else:
            Name = "A"
        #print(Name)
        theRow.append(Name)

        if p.find('td', text='Site Address') !=-1:
            SiteAddress = p.find('td', text='Site Address').find_next('td').text.strip()
        else:
            SiteAddress = "A"
        #print(SiteAddress)
        theRow.append(SiteAddress)

        if p.find('td', text='Legal') !=-1:
            Legal = p.find('td', text='Legal').find_next('td').text.strip()
        else:
            Legal = "A"
        #print(Legal)
        theRow.append(Legal)

#GET DATA FROM TABLE 2--------------------------------------------------------
        if p.find('td', text='Year Built') !=-1 and p.find('td', text='Year Built'):
            YearBuilt = p.find('td', text='Year Built').find_next('td').text.strip()
        else:
            YearBuilt = "0"
        #print(TotalSqFootage)
        theRow.append(YearBuilt)

        if p.find('td', text='Total Square Footage') !=-1 and p.find('td', text='Total Square Footage'):
            TotalSqFootage = p.find('td', text='Total Square Footage').find_next('td').text.strip()
        else:
            TotalSqFootage = "0"
        #print(TotalSqFootage)
        theRow.append(TotalSqFootage)

        if p.find('td', text='Number Of Bedrooms') !=-1 and p.find('td', text='Number Of Bedrooms'):
            NumBedrms = p.find('td', text='Number Of Bedrooms').find_next('td').text.strip()
        else:
            NumBedrms = "0"
        #print(NumBedrms)
        theRow.append(NumBedrms)

        if p.find('td', text='Number Of Baths') !=-1 and p.find('td', text='Number Of Baths'):
            NumBthrms = p.find('td', text='Number Of Baths').find_next('td').text.strip()
        else:
            NumBthrms = "0"
        #print(NumBthrms)
        theRow.append(NumBthrms)

        if p.find('td', text='Grade') !=-1 and p.find('td', text='Grade'):
            Grade = p.find('td', text='Grade').find_next('td').text.strip()
        else:
            Grade = "A"
        #print(Grade)
        theRow.append(Grade)

        if p.find('td', text='Condition') !=-1 and p.find('td', text='Condition'):
            Condition = p.find('td', text='Condition').find_next('td').text.strip()
        else:
            Condition = "A"
        #print(Condition)
        theRow.append(Condition)

        if p.find('td', text='Lot Size') !=-1 and p.find('td', text='Lot Size'):
            LotSize = p.find('td', text='Lot Size').find_next('td').text.strip()
        else:
            LotSize = "0"
        #print(LotSize)
        theRow.append(LotSize)

        if p.find('td', text='Views') !=-1 and p.find('td', text='Views'):
            Views = p.find('td', text='Views').find_next('td').text.strip()
        else:
            Views = "A"
        #print(Views)
        theRow.append(Views)

        if p.find('td', text='Waterfront') !=-1 and p.find('td', text='Waterfront'):
            if len(p.find('td', text='Waterfront').find_next('td').text.strip()) > 0:
                Waterfront = p.find('td', text='Waterfront').find_next('td').text.strip()
            else:
                Waterfront = "No"
        else:
            Waterfront = "A"
        #print(Waterfront) 
        theRow.append(Waterfront)

        # Table1
        # df = pd.DataFrame(columns=list("ABC"))
        Table1.loc[len(Table1)] = theRow



#GET DATA FROM Lower Table------------------------------------------------------
        if p.find('table', id='cphContent_GridViewDBTaxRoll') !=-1 and p.find('table', id='cphContent_GridViewDBTaxRoll'):
            cphContent_GridViewDBTaxRoll = p.find('table', id='cphContent_GridViewDBTaxRoll')

            if cphContent_GridViewDBTaxRoll.find_all('tr') !=-1 and cphContent_GridViewDBTaxRoll.find_all('tr'):
                trs = cphContent_GridViewDBTaxRoll.find_all('tr')
                for tr in trs:
                    if tr.get("class") == "GridViewHeaderStyle":
                        None
                    else:
                        row = []
                        if len([td.get_text(strip=True) for td in tr.find_all('td')]) > 0:  # data row
                            row.append(ParcelNumber)
                            row.extend([td.get_text(strip=True) for td in tr.find_all('td')]) # data row
                            TaxRollTable.loc[len(TaxRollTable)] = row


In [51]:
Table1.to_excel("UpperTable.xlsx")

In [52]:
TaxRollTable.to_excel("TaxRollTable.xlsx")